In [187]:
class WFST:
    def __init__(self):
        self.states = {}
        self.start_state = None
        self.final_states = set()

    def set_start_state(self, state):
        self.start_state = state
        self.add_state(state)

    def add_state(self, state):
        if state not in self.states:
            self.states[state] = {}

    def add_final_state(self, state):
        self.final_states.add(state)
        self.add_state(state)

    def add_transition(self, from_state, to_state, input_symbol, output_symbol, weight=0):
        self.add_state(from_state)
        self.add_state(to_state)
        if input_symbol not in self.states[from_state]:
            self.states[from_state][input_symbol] = []
        self.states[from_state][input_symbol].append((to_state, output_symbol, weight))

    def add_epsilon_transition(self, from_state, to_state, output_symbol, weight=0):
        self.add_transition(from_state, to_state, '', output_symbol, weight)

    def process(self, input_sequence):
        current_states = [(self.start_state, '', 0)]
        print(self.states['0'])
        for symbol in input_sequence:
            next_states = []
            for (state, current_output, current_weight) in current_states:
                if symbol in self.states[state]:
                    for (next_state, output_symbol, weight) in self.states[state][symbol]:
                        next_states.append((next_state, current_output + output_symbol, current_weight + weight))
                if '' in self.states[state]:
                    for (next_state, output_symbol, weight) in self.states[state]['']:
                        next_states.append((next_state, current_output + output_symbol, current_weight + weight))
            current_states = next_states
            #print(current_states)
        
        final_states = [(state, output, weight) for (state, output, weight) in current_states if state in self.final_states]
        if not final_states:
            for (state, current_output, current_weight) in current_states:
                if '' in self.states[state]:
                    for (next_state, output_symbol, weight) in self.states[state]['']:
                        if next_state in self.final_states:
                            final_states.append((next_state, current_output + output_symbol, current_weight + weight))
        return final_states

    def compose(self, other):
        result = WFST()
        result.set_start_state('0')
        i = 0
        for s1 in self.states:
            if s1 not in self.final_states:
                for symbol1 in self.states[s1]:
                        for (n1, o1, w1) in self.states[s1][symbol1]:
                            if symbol1 == '' and n1 in self.final_states:
                                break
                            result.add_transition(chr(i + 48), chr(i + 49), symbol1, o1, w1)
                            i += 1

        print("hi", result.states)
                        
        for s1 in other.states:
            if s1 not in other.final_states:
                for symbol1 in other.states[s1]:
                        for (n1, o1, w1) in other.states[s1][symbol1]:
                            result.add_transition(chr(i + 48), chr(i + 49), symbol1, o1, w1)
                            i += 1
        print("we", result.states)

        result.add_final_state(chr(i + 48))
        
        print(result.states)
        """result.set_start_state((self.start_state, other.start_state))
        for s1 in self.states:
            #print("s1", s1)
            for s2 in other.states:
                #print("s2", s2)
                for symbol1 in self.states[s1]:
                    for (n1, o1, w1) in self.states[s1][symbol1]:
                        if symbol1 == '':
                            result.add_transition((s1, s2), (n1, s2), '', o1, w1)
                        else:
                            for symbol2 in other.states[s2]:
                                for (n2, o2, w2) in other.states[s2][symbol2]:
                                    if symbol2 == '':
                                        result.add_transition((s1, s2), (n1, n2), symbol1, o1, w1 + w2)
                                    else:
                                        result.add_transition((s1, s2), (n1, n2), symbol1 + symbol2, o1 + o2, w1 + w2)
        
        for f1 in self.final_states:
            for f2 in other.final_states:
                result.add_final_state((f1, f2))

        print(result.states)"""

        return result

In [188]:
wfst1 = WFST()
wfst1.set_start_state('0')
wfst1.add_state('1')
wfst1.add_final_state('2')
wfst1.add_transition('0', '1', 'twenty', '2')
wfst1.add_epsilon_transition('1', '2', '0')

"""input_sequence = ['twenty']
result = wfst1.process(input_sequence)
print("Result for ['twenty']:", result)"""

wfst2 = WFST()
wfst2.set_start_state('0')
wfst2.add_final_state('1')
wfst2.add_transition('0', '1', 'one', '1')

wfst_twenties = wfst1.compose(wfst2)

input_sequence_combined = {'twenty', 'one'}
result_combined = wfst_twenties.process(input_sequence_combined)
print("Result for ['twentyone']:", result_combined)

hi {'0': {'twenty': [('1', '2', 0)]}, '1': {}}
we {'0': {'twenty': [('1', '2', 0)]}, '1': {'one': [('2', '1', 0)]}, '2': {}}
{'0': {'twenty': [('1', '2', 0)]}, '1': {'one': [('2', '1', 0)]}, '2': {}}
{'twenty': [('1', '2', 0)]}
Result for ['twentyone']: [('2', '21', 0)]


In [82]:
wfst1 = WFST()
wfst1.set_start_state('q0')
wfst1.add_final_state('q1')
wfst1.add_transition('q0', 'q1', 'twenty', '2')

input_sequence = ['twenty']
result = wfst1.process(input_sequence)
print(result)

wfst2 = WFST()
wfst2.set_start_state('q0')
wfst2.add_final_state('q1')
wfst2.add_transition('q0', 'q1', 'one', '1')

wfst_twenties = wfst1.compose(wfst2)

input_sequence_combined = ['twentyone']
result_combined = wfst_twenties.process(input_sequence_combined)
print("Result for ['twenty', 'one']:", result_combined)

[('q1', '2', 0)]
Result for ['twenty', 'one']: [(('q1', 'q1'), '21', 0)]


In [137]:
class WFST:
    def __init__(self):
        self.states = {}
        self.start_state = None
        self.final_states = set()

    def add_transition(self, from_state, to_state, input_symbol, output_symbol):
        if from_state not in self.states:
            self.states[from_state] = {}
        if input_symbol not in self.states[from_state]:
            self.states[from_state][input_symbol] = []
        self.states[from_state][input_symbol].append((to_state, output_symbol))

    def set_start_state(self, state):
        self.start_state = state

    def add_final_state(self, state):
        self.final_states.add(state)

    def process(self, input_sequence):
        current_states = [(self.start_state, "")]
        for symbol in input_sequence:
            next_states = []
            for (state, output) in current_states:
                if state in self.states and symbol in self.states[state]:
                    for (next_state, output_symbol) in self.states[state][symbol]:
                        next_states.append((next_state, output + output_symbol))
            current_states = next_states
        
        return [output for (state, output) in current_states if state in self.final_states]

    def compose(self, other):
        result = WFST()
        result.set_start_state((self.start_state, other.start_state))
        
        for s1 in self.states:
            for s2 in other.states:
                for symbol in self.states[s1]:
                    for (n1, o1) in self.states[s1][symbol]:
                        if o1 in other.states[s2]:
                            for (n2, o2) in other.states[s2][o1]:
                                result.add_transition((s1, s2), (n1, n2), symbol, o2)
        
        for f1 in self.final_states:
            for f2 in other.final_states:
                result.add_final_state((f1, f2))
        
        return result

# Create WFST for "twenty"
wfst1 = WFST()
wfst1.set_start_state('q0')
wfst1.add_transition('q0', 'q1', 'twenty', '2')
wfst1.add_transition('q1', 'q2', '', '0')
wfst1.add_final_state('q1')
wfst1.add_final_state('q2')

# Test WFST1
print("Result for ['twenty']:", wfst1.process(['twenty']))

# Create WFST for "one"
wfst2 = WFST()
wfst2.set_start_state('q0')
wfst2.add_transition('q0', 'q1', '1', '1')
wfst2.add_final_state('q1')

# Compose WFSTs
wfst_composed = wfst1.compose(wfst2)

# Test composed WFST
print("Result for ['twenty']:", wfst_composed.process(['twenty']))

Result for ['twenty']: ['2']
Result for ['twenty']: []
